<a href="https://colab.research.google.com/github/Natakorn-L/Data-science-bootcamp-8/blob/main/Python/DBD_Web_Scraping%20/DBD_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from IPython.display import display, clear_output
import bs4
import time
import re
import pandas as pd
import numpy as np
from joblib import dump, load

In [ ]:
def company_id_scrape(category):
    try:
        # Open webdriver & enter to DBD website
        driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))
        driver.get('https://datawarehouse.dbd.go.th/')

        # Close ad button
        closebutton= driver.find_element(By.XPATH, '//*[@id="btnWarning"]')
        closebutton.click()
        time.sleep(2)

        # Search for business category
        driver.get(f'https://datawarehouse.dbd.go.th/searchJuristicInfo/{category}/submitObjCode/1')
        time.sleep(5)

        # Wait for the filters button to be clickable
        wait = WebDriverWait(driver, 10)
        filters_button = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[2]/div/div[1]/div[2]/div/button')))

        # Click on the filters button
        filters_button.click()
        time.sleep(2)

        # Filter available status
        status_button= driver.find_element(By.XPATH, '/html/body/div[3]/form/div[2]/div/div/input')
        status_button.click()
        time.sleep(2)
        available = driver.find_element(By.XPATH, '/html/body/div[3]/form/div[2]/div/div[2]/span[1]')
        available.click()
        time.sleep(2)
        search_filter = driver.find_element(By.XPATH, '//*[@id="filterBtn"]')
        search_filter.click()
        time.sleep(2)
        close_filter = driver.find_element(By.XPATH, '/html/body/div[3]/button')
        close_filter.click()
        time.sleep(10)

        # Find a number of total page
        total_page_element = driver.find_element(By.XPATH,'//*[@id="sTotalPage"]')
        # Get the text content of the element
        total_page = total_page_element.text
        # Extract the number from the text content
        total_page = total_page.replace(',', '')
        # Convert the extracted string to an integer
        total_page = int(total_page)


        companies_id = pd.DataFrame()
        # Scrape other pages
        for i in range (1,total_page):
            time.sleep(1)
            clear_output(wait=True)
            print(f'Finished {i} pages from {total_page} pages',end='\r')

            time.sleep(5)
            # Scrape page
            data = driver.page_source
            company_id = pd.read_html(data)[0]
            companies_id = pd.concat([companies_id,company_id],ignore_index=True)
            time.sleep(5)
            # Scroll down the page
            element = driver.find_element(By.XPATH,'//*[@id="next"]')
            driver.execute_script("arguments[0].scrollIntoView(true);", element)
            time.sleep(2)
            # Click next page
            wait = WebDriverWait(driver, 10) # Adjust the timeout as needed
            next_page_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="next"]')))
            next_page_button.click()
            time.sleep(5)
            # Scroll up the page
            element = driver.find_element(By.XPATH,'/html/body/div[2]/div[2]/div/div[1]/div[2]/div/div')
            driver.execute_script("arguments[0].scrollIntoView(true);", element)
            time.sleep(2)
        companies_id.rename(columns={'เลขทะเบียนนิติบุคคล':'ID','ชื่อนิติบุคคล':'Name'},inplace=True)

        return companies_id
    except Exception as e:
        print("An error occurred:", e)
        return None
    finally:
        # Close the webdriver
        driver.quit()

In [ ]:
company_category = 73101
marketing_companies_id = company_id_scrape(company_category)

In [ ]:
# Company detail scraping & Data transform function
def Company_detail_scraping(df='',start=0,end=''):

    csv_df = df
    driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))
    csv_df = csv_df[start:end]
    company_id = list(csv_df['ID'])
    company_name = list(csv_df['Name'])
    company_md = pd.DataFrame()
    balance_sheets = pd.DataFrame()
    finances = pd.DataFrame()
    pnls = pd.DataFrame()

    count = 0
    error_companies = {}

    driver.get('https://datawarehouse.dbd.go.th/')
    closebutton= driver.find_element(By.XPATH, '//*[@id="btnWarning"]')
    closebutton.click()
    time.sleep(5)

    for i, j in zip(company_id, company_name):
        time.sleep(2)
        clear_output(wait=True)

        print(f'ID error list : {error_companies}')
        print(f'Finished {count} companies',end='\r')
        count += 1
        search = driver.find_element(By.XPATH,'//*[@id="key-word"]')
        search.send_keys(i)
        search.send_keys(Keys.ENTER)
        time.sleep(20)
        names = []
        #Company entity
        try:
            data = driver.page_source
            soups = bs4.BeautifulSoup(data, 'html.parser')
            ol_tags = soups.find_all('ol', {'class': 'mt-1'})
            # Loop through each <ol> tag found
            for ol_tag in ol_tags:
                # Find all <li> tags within the current <ol> tag
                li_tags = ol_tag.find_all('li')

                # Loop through each <li> tag found
                for li_tag in li_tags:
                    # Extract the text within the <li> tag and remove any leading/trailing whitespace
                    name = li_tag.get_text(strip=True)

                    # Use regex to remove the forward slash ("/") if present
                    name_without_slash = re.sub(r'/', '', name)

                    # Append the modified name to the list
                    names.append(name_without_slash)

            company_md_loop = {'Name':names}
            company_md_loop = pd.DataFrame(company_md_loop)
            company_md_loop['ID'] = i
            company_md = pd.concat([company_md,company_md_loop],ignore_index=True)

        except Exception as e:
            error = f"Error reading company managing director data: {e}"
            print(error)
            error_companies[i] = str(error)
            time.sleep(2)

        #Company balance sheet
        try:
            balance_sheet = driver.find_element(By.XPATH,'/html/body/div[2]/div[2]/div/div[1]/ul/li[2]/span')
            balance_sheet.click()
            time.sleep(5)
            data = driver.page_source
            time.sleep(2)
            balance_sheet_loop = pd.read_html(data)[0]
            balance_sheet_loop = balance_sheet_loop.set_index('หน่วย : บาท').transpose().reset_index()
            balance_sheet_loop = balance_sheet_loop[balance_sheet_loop['level_1']=='จำนวนเงิน']
            balance_sheet_loop.rename(columns={'level_0':'Year'},inplace=True)
            balance_sheet_loop = balance_sheet_loop.drop('level_1',axis=1)
            balance_sheet_loop.columns = ['Year','ลูกหนี้การค้าสุทธิ','สินค้าคงเหลือ','สินทรัพย์หมุนเวียน','ที่ดิน อาคารและอุปกรณ์','สินทรัพย์ไม่หมุนเวียน','สินทรัพย์รวม','หนี้สินหมุนเวียน','หนี้สินไม่หมุนเวียน','หนี้สินรวม','ส่วนของผู้ถือหุ้น','หนี้สินรวมและส่วนของผู้ถือหุ้น']
            balance_sheet_loop['ID'] = i
            balance_sheet_loop['Name'] = j
            balance_sheets = pd.concat([balance_sheets,balance_sheet_loop],ignore_index=True)
            time.sleep(5)

        except Exception as e:
            error = f"Error reading balance sheet data: {e}"
            print(error)
            error_companies[i] = str(error)

        #Company PNL
        try:
            pnl = driver.find_element(By.XPATH,'/html/body/div[2]/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[1]/div/div/div[1]/div/div[2]/span[2]')
            pnl.click()
            time.sleep(5)
            data = driver.page_source
            time.sleep(2)
            pnl_loop = pd.read_html(data)[0]
            pnl_loop = pnl_loop.set_index('หน่วย : บาท').transpose().reset_index()
            pnl_loop = pnl_loop[pnl_loop['level_1']=='จำนวนเงิน']
            pnl_loop.rename(columns={'level_0':'Year'},inplace=True)
            pnl_loop = pnl_loop.drop('level_1',axis=1)
            pnl_loop.columns = ['Year','รายได้หลัก','รายได้รวม','ต้นทุนขาย','กำไร(ขาดทุน)ขั้นต้น','ค่าใช้จ่ายในการขายและบริหาร','รายจ่ายรวม','ดอกเบี้ยจ่าย','กำไร(ขาดทุน)ก่อนภาษี','ภาษีเงินได้','กำไร(ขาดทุน)สุทธิ']
            pnl_loop['ID'] = i
            pnl_loop['Name'] = j
            pnls = pd.concat([pnls,pnl_loop],ignore_index=True)
        except Exception as e:
            error = f"Error reading PnL data: {e}"
            print(error)
            error_companies[i] = str(error)

        #Financial ratio
        try:
            finance_ratio = driver.find_element(By.XPATH,'/html/body/div[2]/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[1]/div/div/div[1]/div/div[2]/span[3]')
            finance_ratio.click()
            time.sleep(5)
            data = driver.page_source
            finance_loop = pd.read_html(data)[0]
            finance_loop = finance_loop.set_index('อัตราส่วน').transpose().reset_index()
            finance_loop = finance_loop.iloc[1:,:]
            finance_loop = finance_loop.drop(columns=['level_1','level_2','level_3','level_4'],axis=1)
            finance_loop.columns = [
                            'Year',
                            'อัตราผลตอบแทนจากสินทรัพย์รวม(ROA)(%)',
                            'อัตราผลตอบแทนจากส่วนของผู้ถือหุ้น(ROE)(%)',
                            'ผลตอบแทนจากกำไรขั้นต้นต่อรายได้รวม(%)',
                            'ผลตอบแทนจากการดำเนินงานต่อรายได้รวม (%)',
                            'ผลตอบแทนจากกำไรสุทธิต่อรายได้รวม (%)',
                            'อัตราส่วนทุนหมุนเวียน(เท่า)',
                            'อัตราการหมุนเวียนของลูกหนี้(เท่า)',
                            'อัตราการหมุนเวียนของสินค้าคงเหลือ(เท่า)',
                            'อัตราการหมุนเวียนของเจ้าหนี้(เท่า)',
                            'อัตราการหมุนเวียนของสินทรัพย์รวม(เท่า)',
                            'อัตราค่าใช้จ่ายการดำเนินงานต่อรายได้รวม(%)',
                            'อัตราส่วนสินทรัพย์รวมต่อส่วนของผู้ถือหุ้น(เท่า)',
                            'อัตราส่วนหนี้สินรวมต่อสินทรัพย์รวม(เท่า)',
                            'อัตราส่วนหนี้สินรวมต่อส่วนของผู้ถือหุ้น(เท่า)',
                            'อัตราส่วนหนี้สินรวมต่อทุนดำเนินงาน(เท่า)'
                            ]
            finance_loop['ID'] = i
            finance_loop['Name'] = j
            finances = pd.concat([finances,finance_loop],ignore_index=True)
            time.sleep(3)
        except Exception as e:
            error = f"Error reading Financial ratio data: {e}"
            print(error)
            error_companies[i] = str(error)

        driver.get('https://datawarehouse.dbd.go.th/')
        time.sleep(3)

    #Select relavant columns
    balance_sheets = balance_sheets[['ID','Name','Year','ลูกหนี้การค้าสุทธิ','สินค้าคงเหลือ','สินทรัพย์หมุนเวียน','ที่ดิน อาคารและอุปกรณ์','สินทรัพย์ไม่หมุนเวียน','สินทรัพย์รวม','หนี้สินหมุนเวียน','หนี้สินไม่หมุนเวียน','หนี้สินรวม','ส่วนของผู้ถือหุ้น','หนี้สินรวมและส่วนของผู้ถือหุ้น']]
    pnls = pnls[['ID','Name','Year','รายได้หลัก','รายได้รวม','ต้นทุนขาย','กำไร(ขาดทุน)ขั้นต้น','ค่าใช้จ่ายในการขายและบริหาร','รายจ่ายรวม','ดอกเบี้ยจ่าย','กำไร(ขาดทุน)ก่อนภาษี','ภาษีเงินได้','กำไร(ขาดทุน)สุทธิ']]
    finances = finances[
                            [
                                'ID',
                                'Name',
                                'Year',
                                'อัตราผลตอบแทนจากสินทรัพย์รวม(ROA)(%)',
                                'อัตราผลตอบแทนจากส่วนของผู้ถือหุ้น(ROE)(%)',
                                'ผลตอบแทนจากกำไรขั้นต้นต่อรายได้รวม(%)',
                                'ผลตอบแทนจากการดำเนินงานต่อรายได้รวม (%)',
                                'ผลตอบแทนจากกำไรสุทธิต่อรายได้รวม (%)',
                                'อัตราส่วนทุนหมุนเวียน(เท่า)',
                                'อัตราการหมุนเวียนของลูกหนี้(เท่า)',
                                'อัตราการหมุนเวียนของสินค้าคงเหลือ(เท่า)',
                                'อัตราการหมุนเวียนของเจ้าหนี้(เท่า)',
                                'อัตราการหมุนเวียนของสินทรัพย์รวม(เท่า)',
                                'อัตราค่าใช้จ่ายการดำเนินงานต่อรายได้รวม(%)',
                                'อัตราส่วนสินทรัพย์รวมต่อส่วนของผู้ถือหุ้น(เท่า)',
                                'อัตราส่วนหนี้สินรวมต่อสินทรัพย์รวม(เท่า)',
                                'อัตราส่วนหนี้สินรวมต่อส่วนของผู้ถือหุ้น(เท่า)',
                                'อัตราส่วนหนี้สินรวมต่อทุนดำเนินงาน(เท่า)'
                            ]
                        ]

    driver.close()
    print('Finish all companies yeah!!!')
    print(f'ID error list : {error_companies}')

    data_to_save = {'MD Name':company_md,'Balance Sheet':balance_sheets,'Profit & Loss':pnls,'Financial Ratio':finances,'Error Company ID':error_companies}
    file_path = f"C:\\Users\\HP\\Downloads\\{company_category}_Company_{start}_{end}.joblib"
    dump(data_to_save, file_path)
    loaded_data = load(file_path)

    return  loaded_data

In [ ]:
loaded_data = Company_detail_scraping(marketing_companies_id,0,1000)

In [ ]:
# MD name cleaning function
def MD_name_clean(name):

    # Remove "เงินสด"
    name = re.sub(r'\s*เงินสด\s*', '', name)

    # Remove floating point numbers
    name = re.sub(r'\d+,?\d*\.\d+', '', name)

    # Remove "บาท"
    name = re.sub(r'\s*บาท', '', name)

    # Remove "ลงหุ้นด้วยจำนวน"

    name = re.sub(r'\s*ลงหุ้นด้วยจำนวน\s*','',name)

    name = re.sub(r'\s*1,\s*','',name)

    name = re.sub(r'\s*จำนวน\s*','',name)

    # Remove leading/trailing whitespace
    name = name.strip()

    return name

In [ ]:
loaed_data = Company_detail_scraping(marketing_companies_id,0,100)
company_md = loaed_data['MD Name']
company_md['Name'] = company_md['Name'].apply(MD_name_clean)